In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud
import requests # librería para manejar solicitudes
# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans
# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # librería para graficar mapas 

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing th

# PARTE 1

### Cargar los datos de los codigos postales desde la pagina de wikipedia

In [3]:
df_toronto = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1008658788') 
df_toronto = df_toronto[0]
df_toronto.head(5)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Procese únicamente las celdas que tengan un municipio asignado. Ignore las celdas con un municipio que esté No asignado.

In [4]:
df_toronto = df_toronto[df_toronto['Borough'] != 'Not assigned']
df_toronto = df_toronto.reset_index(drop=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Si una celda tiene un municipio, pero un vecindario No asignado, entonces el vecindario será el mismo que el municipio.

In [5]:
df_toronto['Neighbourhood'] = np.where(df_toronto.Neighbourhood == 'Not assigned', df_toronto.Borough, df_toronto.Neighbourhood)
df_toronto.head(5)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df_toronto.shape

(103, 3)

Tamaño final del Dataframe es de 103 filas y 3 columnas

# PARTE 2

Agregar las coordenadas geoespaciales al dataframe

Cargar coordenadas

In [7]:
df_Postal = pd.read_csv("https://cocl.us/Geospatial_data")
df_Postal.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Unir dataframes con nombres y el de coordenadas geoespaciales

In [8]:
df = pd.merge(df_toronto, df_Postal, on = 'Postal Code')
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# PARTE 3 

### Primero filtrar. Dejar solo los vecindarios de Toronto

In [9]:
filtro = 'Toronto'
df_toronto = df[df.Borough.str.contains(filtro, case=False)].reset_index(drop=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Crear funcion que trae de foursquare los primeros 50 venues mas cercanos

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    #
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = "https://api.foursquare.com/v3/places/search"

        params = {
            "ll": str(lat) + "," + str(lng),
            "limit": 50, # Maximo permitido por foursquare en cuentas gratuitas
            "sort": "DISTANCE"
        }

        headers = {
            "Accept": "application/json",
            "Authorization": "fsq3ArgH46bp1yN5Llr6X5BvyYTl+N1RtjmbnnUTRj6V9Dc="
        }
        #
        # solicitud GET
        response = requests.request("GET", url, params=params, headers=headers)
        #
        results = response.json()
        if len(results)>0:
            jsontxt = results['results']
            df_results = pd.json_normalize(jsontxt)
            #
            categories = pd.json_normalize(df_results['categories']) # objeto JSON
            categories_names = pd.json_normalize(categories[0]) # obj[0]['name'].head()
            categories_names.rename(columns = {'name':'categorie_name'}, inplace = True)
            categories_names.insert(0, 'New_Index', range(0, 0 + len(categories_names)))
            #
            venues = df_results
            venues.insert(0, 'New_Index', range(0, 0 + len(venues)))
            #
            df_venues = pd.merge(venues, categories_names, on='New_Index', how='inner')
            #
            filtered_columns = ['name', 'categorie_name', 'geocodes.main.latitude','geocodes.main.longitude']
            #
            df_venues = df_venues[filtered_columns]
            #
            # regresa solo información relevante de cada sitio cercano
            for i in range(len(df_venues)):
                venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    df_venues.iloc[i]['name'],
                    df_venues.iloc[i]['geocodes.main.latitude'],
                    df_venues.iloc[i]['geocodes.main.longitude'],
                    df_venues.iloc[i]['categorie_name'])])

            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            nearby_venues.columns = ['Neighborhood', 
                          'Neighborhood Latitude', 
                          'Neighborhood Longitude', 
                          'Venue', 
                          'Venue Latitude', 
                          'Venue Longitude', 
                          'Venue Category']
    
    return(nearby_venues)

#### Ahora llenar todas las venues para cada vecindario. Todo de una vez

In [11]:
toronto_venues = []
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
toronto_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656146,-79.357052,Restaurant
1,"Regent Park, Harbourfront",43.65426,-79.360636,Soma Chocolatemaker,43.650730,-79.358203,Restaurant
2,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650317,-79.359259,Monument
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cluny Bistro,43.650476,-79.357835,Bistro
4,"Regent Park, Harbourfront",43.65426,-79.360636,Rooster Coffee House,43.651790,-79.365290,Coffee Shop


#### Encontremos cuantas categorías únicas se pueden conservar de todos los sitios regresados

In [12]:
print('Hay {} categorias unicas.'.format(len(toronto_venues['Venue Category'].unique())))

Hay 103 categorias unicas.


#### Analizar cada vecindario de Toronto

In [13]:
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
toronto_onehot['Neighborhood'] = [''.join(map(str, l)) for l in toronto_venues['Neighborhood']]

# mover la columna de barrio a la primer columna
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Arts and Entertainment,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Barbershop,...,Stadium,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Urban Park,Vegan and Vegetarian Restaurant,Vietnamese Restaurant,Wine Bar
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Agrupemos las filas por barrios tomando la média de la frecuencia de la ocurrencia de cada categoría

In [14]:
toronto_grouped = toronto_onehot.groupby(toronto_onehot['Neighborhood']).mean().reset_index()
toronto_grouped.head()

,Neighborhood,American Restaurant,Art Gallery,Arts and Entertainment,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Barbershop,...,Stadium,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Urban Park,Vegan and Vegetarian Restaurant,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.02,0.02,0.00,0.0,0.00,0.00,0.00,0.02,0.0,...,0.0,0.00,0.0,0.00,0.02,0.04,0.0,0.00,0.00,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.00,0.00,0.0,0.02,0.04,0.00,0.06,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.02,0.02,0.04
2,"Business reply mail Processing Centre, South C...",0.02,0.00,0.00,0.0,0.00,0.00,0.02,0.04,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.02,0.02,0.00,0.0,0.02,0.04,0.00,0.04,0.0,...,0.0,0.00,0.0,0.00,0.02,0.04,0.0,0.00,0.00,0.02
4,Central Bay Street,0.02,0.02,0.02,0.0,0.00,0.02,0.00,0.02,0.0,...,0.0,0.02,0.0,0.02,0.02,0.00,0.0,0.00,0.00,0.00


#### Escribamos una función para ordenar los sitios en orden descendente.

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Generemos el nuevo dataframe y mostremos los primeros 10 sitios de cada barrio.

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Venue mas comun'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Venue mas comun'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Venue mas comun,2nd Venue mas comun,3rd Venue mas comun,4th Venue mas comun,5th Venue mas comun,6th Venue mas comun,7th Venue mas comun,8th Venue mas comun,9th Venue mas comun,10th Venue mas comun
0,Berczy Park,Café,Music Venue,Park,Bistro,Restaurant,Coffee Shop,Theater,Pizzeria,Clothing Store,Dance Studio
1,"Brockton, Parkdale Village, Exhibition Place",Cocktail Bar,Pizzeria,Park,Bar,Korean Restaurant,Restaurant,Café,Italian Restaurant,Wine Bar,Bakery
2,"Business reply mail Processing Centre, South C...",Café,Park,Coffee Shop,Restaurant,Bistro,Pizzeria,Bar,Grocery Store / Supermarket,Furniture and Home Store,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Park,Pizzeria,Bookstore,Theater,Bakery,Bar,Music Venue,Lounge,Museum,Movie Theater
4,Central Bay Street,Café,Clothing Store,Pizzeria,Park,Hair Salon,Grocery Store / Supermarket,Music Venue,Furniture and Home Store,Coffee Shop,Shoe Store


#### Ejecutemos k-means para agrupar los barrios en 5 agrupaciones.

In [17]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_[0:10]

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` b

array([3, 1, 0, 3, 4, 1, 4, 3, 2, 2], dtype=int32)

#### Generemos un nuevo dataframe que incluya la agrupación asi como los 10 sitios mas populares de cada barrio.

In [18]:
# añadir etiquetas

df_toronto.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# juntar manhattan_grouped con manhattan_data 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # revisar las ultimas columnas

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Venue mas comun,2nd Venue mas comun,3rd Venue mas comun,4th Venue mas comun,5th Venue mas comun,6th Venue mas comun,7th Venue mas comun,8th Venue mas comun,9th Venue mas comun,10th Venue mas comun
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Café,Park,Coffee Shop,Restaurant,Clothing Store,Bistro,Furniture and Home Store,Playground,Museum,Farmers' Market
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Café,Clothing Store,Music Venue,Bakery,Bar,Movie Theater,Pizzeria,Grocery Store / Supermarket,Shoe Store,Coffee Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Café,Pizzeria,Clothing Store,Music Venue,Coffee Shop,Hair Salon,Park,Retail,Record Store,Drugstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Café,Park,Clothing Store,Coffee Shop,Restaurant,Pizzeria,Bar,Bistro,Music Venue,Furniture and Home Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Café,Park,Restaurant,Coffee Shop,Bar,Brewery,Grocery Store / Supermarket,Farmers' Market,Pizzeria,Japanese Restaurant


#### Visualicemos las agrupaciones en un mapa

In [20]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Toronto_explrer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# crear mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# FIN